# SEPTA Data Project
#### William McKee
#### December 2017

SEPTA is a public agency responsible for the public transportation system in Philadelphia and its Pennsylvania suburbs.  SEPTA stands for Southeastern Pennsylvania Transportation Authority. 

This code analyzes the data set for SEPTA Bus and Rail lines downloaded from https://transitfeeds.com.  I downloaded the SEPTA Bus zip file and renamed gfts.zip to septa_bus_gfts.zip.  I downloaded the SEPTA Rail zip file and renamed gfts.zip to septa_rail_gfts.zip.

## Data Set Conversion

The code below checks the contents of both zip files, displays some zip file contents, and converts the files to csv format.

In [1]:
import zipfile
import csv
import os

def read_and_print_first_lines_from_zipped_file(zipfilename, limit):
    """
    Reads zip file and prints the first limit lines from each file contained in the zip file
    zipfilename = zip file name (such as 'example.zip')
    limit = number of lines to print in file
    """
    print()
    print("CONTENTS OF ZIP FILE " + zipfilename + ":")
    print()
    with zipfile.ZipFile(zipfilename, 'r') as z:
        file_name_list = sorted(z.namelist())
        for file in file_name_list:
            print(file)
            with z.open(file, 'r') as input_file:
                for line_number, line in enumerate(input_file):
                    if line_number > limit:
                        break
                    print(line)
            print()
    print()

# Loop through zip files
NUM_LINES = 5
ZIP_FILE_NAMES = ['septa_bus_gfts.zip', 'septa_rail_gfts.zip']
DIRECTORY_NAMES = []
for file in ZIP_FILE_NAMES:
    # Read the zip files and display some file contents
    read_and_print_first_lines_from_zipped_file(file, NUM_LINES)

    # Extract zip file contents
    directory_name = os.path.splitext(file)[0]
    DIRECTORY_NAMES.append(directory_name)
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(directory_name)

    # Convert txt files to csv files
    os.chdir(directory_name)
    for input_file in os.listdir('.'):
        with open(input_file, 'r') as in_file:
            stripped = (line.strip() for line in in_file)
            lines = (line.split(",") for line in stripped if line)
            output_file = os.path.splitext(input_file)[0] + ".csv"
            print("Convert " + input_file + " contents to " + output_file)
            with open(output_file, 'w', ) as out_file:
                writer = csv.writer(out_file, lineterminator = '\n')
                writer.writerows(lines)
            
    # Remove original text files
    for item in os.listdir('.'):
        if item.endswith(".txt"):
            os.remove(item)

    os.chdir('..')


CONTENTS OF ZIP FILE septa_bus_gfts.zip:

agency.txt
b'agency_name,agency_url,agency_timezone,agency_lang,agency_fare_url\r\n'
b'SEPTA,http://www.septa.org,America/New_York,EN,http://www.septa.org/fares/transit/index.html'

calendar.txt
b'service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date\r\n'
b'10,1,1,1,1,1,0,0,20170903,20180224\r\n'
b'11,0,0,0,0,0,0,0,20170903,20180224\r\n'
b'12,0,0,0,0,0,1,0,20170903,20180224\r\n'
b'13,0,0,0,0,0,0,1,20170903,20180224\r\n'
b'16,1,1,1,1,1,0,0,20170903,20180224\r\n'

calendar_dates.txt
b'service_id,date,exception_type\r\n'
b'10,20170904,2\r\n'
b'13,20170904,1\r\n'
b'16,20170904,2\r\n'
b'19,20170904,1\r\n'
b'22,20170904,2\r\n'

fare_attributes.txt
b'fare_id,price,currency_type,payment_method,transfers,transfer_duration\r\n'
b'1,2.50,USD,0,0,0\r\n'
b'2,3.50,USD,0,1,3600\r\n'
b'3,4.50,USD,0,2,3600\r\n'
b'13,7.00,USD,0,0,0\r\n'
b'14,8.00,USD,0,1,3600\r\n'

fare_rules.txt
b'fare_id,origin_id,destination_id\r\n'
b'1,1,1\

## Data Set Basics

We explore our CSV files for both bus and rail.  The groupings occur only on specific files which will reveal more useful information.

In [2]:
import pandas as pd

# Print sizes of CSV files
for directory in DIRECTORY_NAMES:
    os.chdir(directory)
    print("Looking at " + directory + " contents")
    print()
    for input_file in os.listdir('.'):
        print("Description of " + input_file + ":")
        data_set = pd.read_csv(input_file)
        print(data_set.shape)
        print()
    os.chdir('..')

Looking at septa_bus_gfts contents

Description of agency.csv:
(1, 5)

Description of calendar.csv:
(28, 10)

Description of calendar_dates.csv:
(168, 3)

Description of fare_attributes.csv:
(6, 6)

Description of fare_rules.csv:
(9, 3)

Description of routes.csv:
(139, 7)

Description of shapes.csv:
(570717, 4)

Description of stops.csv:
(13701, 8)

Description of stop_times.csv:
(3121225, 5)

Description of transfers.csv:
(1, 4)

Description of trips.csv:
(52000, 7)

Looking at septa_rail_gfts contents

Description of agency.csv:
(1, 6)

Description of calendar.csv:
(5, 10)

Description of calendar_dates.csv:
(2, 3)

Description of fare_attributes.csv:
(0, 6)

Description of fare_rules.csv:
(0, 5)

Description of routes.csv:
(13, 9)

Description of shapes.csv:
(180235, 4)

Description of stops.csv:
(155, 7)

Description of stop_times.csv:
(25082, 7)

Description of transfers.csv:
(3, 3)

Description of trips.csv:
(1711, 8)



In [3]:
# Groupby could provide useful information for some files
GROUPBY_FILES_FIELDS = {'shapes.csv': 'shape_id', 
                        'trips.csv': 'route_id', 
                        'stop_times.csv': 'trip_id'}
for directory in DIRECTORY_NAMES:
    os.chdir(directory)
    print("Looking at " + directory + " groupby contents")
    print()
    for input_file in os.listdir('.'):
        if (input_file in GROUPBY_FILES_FIELDS.keys()):
            this_field = GROUPBY_FILES_FIELDS[input_file]
            print("Description of " + input_file + " Groupings:")
            data_set = pd.read_csv(input_file)
            data_set_distinct = data_set.groupby(this_field)[this_field].count()
            print(data_set_distinct)
            print()
    os.chdir('..')

Looking at septa_bus_gfts groupby contents

Description of shapes.csv Groupings:
shape_id
203286     296
203287     296
203288     280
203290     195
203291     273
203292     188
203293     289
203305     211
203307     189
203308     204
203310     182
203311      95
203312     288
203313      84
203314      84
203315     338
203316     790
203317     896
203318     790
203319     900
203320     872
203322     305
203323     372
203324     387
203325     790
203326     306
203327     321
203329     381
203330     361
203332     814
          ... 
206251     328
206287     253
206288     276
206289     170
206290     249
206291     249
206293     235
206294     257
206305     251
206308     403
206309     578
206310     484
206311    1180
206312     496
206313    1086
206314     597
206315     409
206317     304
206318     381
206319    1001
206320     990
206321     408
206322     370
206323    1079
206324     457
206325     366
206326     884
206327    1337
206329    1419
208290    

## Train Routes

Here, I will get the data associated with various train routes (such as Trenton Rail Line for train #734).

In [4]:
# Train directory
os.chdir(DIRECTORY_NAMES[1])

# List route information
routes_data_set = pd.read_csv('routes.csv')

print("Train Routes Data Set")
print(routes_data_set.to_string(columns=['route_id', 'route_short_name', 'route_color'], index=False))

Train Routes Data Set
route_id          route_short_name route_color
    AIR              Airport Line      91456C
    CHE   Chestnut Hill East Line      94763C
    CHW   Chestnut Hill West Line      00B4B2
    LAN  Lansdale/Doylestown Line      775B49
    MED          Media/Elwyn Line      007CC8
    FOX            Fox Chase Line      FF823D
    NOR  Manayunk/Norristown Line      EE4C69
    PAO      Paoli/Thorndale Line      20825C
    CYN               Cynwyd Line      6F549E
    TRE              Trenton Line      F683C9
    WAR           Warminster Line      F7AF42
    WIL    Wilmington/Newark Line      8AD16B
    WTR         West Trenton Line      5D5EBC


In [5]:
# Find trips associated with different train lines
train_lines = ['TRE', 'WTR', 'FOX']

def get_train_line_data(route_id):
    '''
    Returns data set associated with a particular train route
    route_id = train line route Id code (such as TRE)
    '''
    trips_data_set = pd.read_csv('trips.csv')
    trips_data_set = trips_data_set.loc[trips_data_set['route_id'] == route_id]
    return trips_data_set


def print_train_line_data(route_id):
    '''
    Prints data associated with a particular train route
    route_id = train line route Id code (such as TRE)
    '''
    trips_data_set = get_train_line_data(route_id)

    print("Trips Data Set for rail " + route_id + ":")
    print(trips_data_set.to_string(columns=['trip_id', 'service_id', 'trip_headsign', 'block_id', 'shape_id'], 
                                   index=False, justify='left'))
    
for line in train_lines:
    print_train_line_data(line)
    print()
    print()

Trips Data Set for rail TRE:
trip_id         service_id trip_headsign              block_id  shape_id
 TRE_717_V77_M  M4                          Trenton   717        7701  
 TRE_723_V77_M  M4                          Trenton   723        7701  
  TRE_773_V5_M  M1                          Trenton   773        7701  
  TRE_705_V5_M  M1                          Trenton   705        7701  
 TRE_9741_V5_M  M1                          Trenton  9741        7701  
 TRE_711_V66_M  M3                          Trenton   711        7701  
 TRE_7218_V5_M  M1         Center City Philadelphia  7218      701007  
 TRE_708_V77_M  M4         Center City Philadelphia   708      701004  
 TRE_1766_V5_M  M1         Center City Philadelphia  1766      701005  
  TRE_774_V5_M  M1         Center City Philadelphia   774      701004  
 TRE_722_V77_M  M4         Center City Philadelphia   722      701004  
 TRE_7406_V5_M  M1         Center City Philadelphia  7406      701004  
 TRE_9737_V5_M  M1                

In [6]:
# Find trips associated with individual trains
train_trips = [[['TRE'], [ 734,  735], [ 'TRE_734_V5_M', 'TRE_735_V5_M']], 
               [['WTR'], [ 361, 6370], [ 'WTR_361_V5_M', 'WTR_6370_V55_M']], 
               [['FOX'], [8749, 7848], ['FOX_8749_V5_M', 'FOX_7848_V1_M']]]

def get_train_car_data(route_id, train_id):
    '''
    Returns data set associated with a particular train car
    route_id = train line route Id code (such as TRE)
    train_id = train car Id code (such as 734)
    '''
    # Get route data
    trips_data_set = get_train_line_data(route_id)
    
    # Get train car data
    train_data_set = trips_data_set.loc[trips_data_set['block_id'] == train_id]
    return train_data_set
    

def print_train_car_data(route_id, train_id):
    '''
    Prints data associated with a particular train car
    route_id = train line route Id code (such as TRE)
    train_id = train car Id code (such as 734)
    '''
    # Get train car data
    train_data_set = get_train_car_data(route_id, train_id)

    print("Trips Data Set for rail " + route_id + " Train #" + str(train_id) + ":")
    print(train_data_set.to_string(columns=['route_id', 'service_id', 'trip_id', 'trip_headsign', 'block_id', 'shape_id'], 
                                   index=False, justify='left'))

for route_id_list, train_id_list, trip_id_list in train_trips:
    route_id = route_id_list[0]
    for train_id in train_id_list:
        print_train_car_data(route_id, train_id)
        print()
        print()

Trips Data Set for rail TRE Train #734:
route_id service_id trip_id        trip_headsign              block_id  shape_id
TRE      M3         TRE_734_V66_M  Center City Philadelphia  734       701004  
TRE      M1          TRE_734_V5_M  Center City Philadelphia  734       701004  
TRE      M4         TRE_734_V77_M  Center City Philadelphia  734       701004


Trips Data Set for rail TRE Train #735:
route_id service_id trip_id        trip_headsign  block_id  shape_id
TRE      M1          TRE_735_V5_M  Trenton       735       7701    
TRE      M3         TRE_735_V66_M  Trenton       735       7701    
TRE      M4         TRE_735_V77_M  Trenton       735       7701


Trips Data Set for rail WTR Train #361:
route_id service_id trip_id       trip_headsign              block_id  shape_id
WTR      M1         WTR_361_V5_M  Center City Philadelphia  361       327007


Trips Data Set for rail WTR Train #6370:
route_id service_id trip_id         trip_headsign  block_id  shape_id
WTR      M1       

In [7]:
# Obtain the schedule for individual trains
def get_schedule(route_id, train_id, trip_id):
    '''
    Returns schedule data set associated with a particular train car
    route_id = train line route Id code (such as TRE)
    train_id = train car Id code (such as 734)
    trip_id  = trip Id code (such as TRE_734_V5_M)
    '''
    # Get train car data
    train_data_set = get_train_car_data(route_id, train_id)
    
    # Get stop times data set
    stop_times_data_set = pd.read_csv('stop_times.csv')
    stop_times_data_set = stop_times_data_set[stop_times_data_set['trip_id'] == trip_id]
    return stop_times_data_set
    
def print_schedule(route_id, train_id, trip_id):
    '''
    Prints schedule associated with a particular train car
    route_id = train line route Id code (such as TRE)
    train_id = train car Id code (such as 734)
    trip_id  = trip Id code (such as TRE_734_V5_M)
    '''
    # Get stop times data set
    stop_times_data_set = get_schedule(route_id, train_id, trip_id)

    print("Stop Times Data Set for rail " + route_id + " Train #" + str(train_id) + " for Trip " + trip_id + ":")
    print(stop_times_data_set.to_string(columns=['trip_id', 'arrival_time', 'stop_id', 'stop_sequence'], 
                                       index=False, justify='left'))
        
for route_id_list, train_id_list, trip_id_list in train_trips:
    route_id = route_id_list[0]
    for train_id, trip_id in zip(train_id_list, trip_id_list):
        print_schedule(route_id, train_id, trip_id)
        print()
        print()

Stop Times Data Set for rail TRE Train #734 for Trip TRE_734_V5_M:
trip_id       arrival_time  stop_id  stop_sequence
TRE_734_V5_M  10:43:00     90701     1           
TRE_734_V5_M  10:50:00     90702     4           
TRE_734_V5_M  10:54:00     90703     6           
TRE_734_V5_M  10:58:00     90704     7           
TRE_734_V5_M  11:00:00     90705     8           
TRE_734_V5_M  11:02:00     90706     9           
TRE_734_V5_M  11:05:00     90707    11           
TRE_734_V5_M  11:09:00     90708    12           
TRE_734_V5_M  11:10:00     90709    13           
TRE_734_V5_M  11:13:00     90710    15           
TRE_734_V5_M  11:20:00     90711    17           
TRE_734_V5_M  11:33:00     90004    27


Stop Times Data Set for rail TRE Train #735 for Trip TRE_735_V5_M:
trip_id       arrival_time  stop_id  stop_sequence
TRE_735_V5_M  12:55:00     90007    18           
TRE_735_V5_M  13:01:00     90006    21           
TRE_735_V5_M  13:06:00     90005    23           
TRE_735_V5_M  13:10:00 

In [8]:
pd.options.mode.chained_assignment = None  # default='warn'

# Obtain the stops for individual trains
def get_stops(route_id, train_id, trip_id):
    '''
    Returns stops data set associated with a particular train car
    route_id = train line route Id code (such as TRE)
    train_id = train car Id code (such as 734)
    trip_id  = trip Id code (such as TRE_734_V5_M)
    '''
    # Get schedule data set
    stop_times_data_set = get_schedule(route_id, train_id, trip_id)
    
    # Obtain the list of stops for this train
    stop_sequence_ids = dict(zip(stop_times_data_set.stop_id, stop_times_data_set.stop_sequence))
    
    # Get stops data set
    stops_data_set = pd.read_csv('stops.csv')
    stops_data_set = stops_data_set[stops_data_set['stop_id'].isin(stop_sequence_ids.keys())]
    
    # Add sequence id to stops data set
    sequence_ids = []
    for stop_id in stops_data_set['stop_id']:
        sequence_ids.append(stop_sequence_ids[stop_id])
    stops_data_set['sequence_id'] = sequence_ids
    stops_data_set.sort_values('sequence_id',inplace=True)
    
    return stops_data_set


def print_stops(route_id, train_id, trip_id):
    '''
    Prints stops associated with a particular train car
    route_id = train line route Id code (such as TRE)
    train_id = train car Id code (such as 734)
    trip_id  = trip Id code (such as TRE_734_V5_M)
    '''
    # Get stops data set
    stops_data_set = get_stops(route_id, train_id, trip_id)
    
    print("Stops Data Set for rail " + route_id + " Train #" + str(train_id) + " for Trip " + trip_id + ":")
    print(stops_data_set.to_string(columns=['sequence_id', 'stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'zone_id'], 
                                   index=False, justify='left'))
    
for route_id_list, train_id_list, trip_id_list in train_trips:
    route_id = route_id_list[0]
    for train_id, trip_id in zip(train_id_list, trip_id_list):
        print_stops(route_id, train_id, trip_id)
        print()
        print()

Stops Data Set for rail TRE Train #734 for Trip TRE_734_V5_M:
sequence_id  stop_id stop_name                   stop_lat   stop_lon  zone_id
 1           90701                      Trenton  40.217778 -74.755000  NJ    
 4           90702          Levittown-Tullytown  40.140278 -74.816944   4    
 6           90703                      Bristol  40.104722 -74.854722   4    
 7           90704                      Croydon  40.093611 -74.906667   3    
 8           90705                    Eddington  40.083056 -74.933611   3    
 9           90706            Cornwells Heights  40.071667 -74.952222   3    
11           90707                   Torresdale  40.054444 -74.984444   3    
12           90708               Holmesburg Jct  40.032778 -75.023611   2    
13           90709                       Tacony  40.023333 -75.038889   2    
15           90710                   Bridesburg  40.010556 -75.069722   2    
17           90711    North Philadelphia Amtrak  39.997222 -75.155000   1    
27

## Bus Routes

Now it's time to explore Bus Routes (such as the #28 Bus which runs in northeast Philadelphia).

In [9]:
# Bus directory
os.chdir('..')
os.chdir(DIRECTORY_NAMES[0])

# List route information
routes_data_set = pd.read_csv('routes.csv')

print("Routes Data Set")
print(routes_data_set.to_string(columns=['route_id', 'route_long_name', 'route_type'], index=False))

Routes Data Set
route_id                      route_long_name  route_type
      1             Parx Casino to 54th-City           3
     10          13th-Market to 63rd-Malvern           0
    101                  Media to 69th St TC           0
    102            Sharon Hill to 69th St TC           0
    103                Ardmore to 69th St TC           3
    104         West Chester U to 69th St TC           3
    105               Rosemont to 69th St TC           3
    106                  Paoli to 69th St TC           3
    107      Lawrence Park to 69th St TC-107           3
    108         UPS or Airport to 69th St TC           3
    109             Chester TC to 69th St TC           3
     11      13th-Market to Darby Trans Cntr           0
    110           Penn State U to 69th St TC           3
    111            Chadds Ford to 69th St TC           3
    112                   DCCC to 69th St TC           3
    113         Tri State Mall to 69th St TC           3
    114       

In [10]:
# Find trips associated with different bus lines
bus_lines = ['28', '70', '88']

def get_bus_line_data(route_id):
    '''
    Returns data set associated with a particular bus route
    route_id = bus line route Id code (such as 28)
    '''
    trips_data_set = pd.read_csv('trips.csv')
    trips_data_set = trips_data_set.loc[trips_data_set['route_id'] == route_id]
    return trips_data_set


def print_bus_line_data(route_id):
    '''
    Prints data associated with a particular bus route
    route_id = train line route Id code (such as TRE)
    '''
    trips_data_set = get_train_line_data(route_id)

    print("Trips Data Set for bus " + route_id + ":")
    print(trips_data_set.to_string(columns=['trip_id', 'service_id', 'trip_headsign', 'block_id', 'shape_id'], 
                                   index=False, justify='left'))
    
for line in bus_lines:
    print_bus_line_data(line)
    print()
    print()

Trips Data Set for bus 28:
trip_id  service_id trip_headsign                      block_id  shape_id
45049    10          "Fern Rock Transportation Center"  1308     205690  
45050    10          "Fern Rock Transportation Center"  1312     205690  
45051    10          "Fern Rock Transportation Center"  1313     205690  
45052    10          "Fern Rock Transportation Center"  1215     205690  
45054    10          "Fern Rock Transportation Center"  1316     205690  
45055    10          "Fern Rock Transportation Center"  1315     205690  
45056    10          "Fern Rock Transportation Center"  1314     205690  
45057    10          "Fern Rock Transportation Center"  1312     205690  
45058    10          "Fern Rock Transportation Center"  1230     205690  
45059    10          "Fern Rock Transportation Center"  1315     205690  
45060    10          "Fern Rock Transportation Center"  1314     205690  
45061    10          "Fern Rock Transportation Center"  1312     205690  
45062    10

In [11]:
# Find trips associated with individual buses
bus_trips = [[['28'], ['1308', '1309'], ['45070', '45160']], 
             [['70'], ['1225', '1226'], ['67067', '67255']], 
             [['88'], ['2511', '2512'], ['70400', '70499']]]

def get_bus_data(route_id, bus_id):
    '''
    Returns data set associated with a particular bus service
    route_id = bus line route Id code (such as 28)
    bus_id   = bus Id code (such as 1308)
    '''
    # Get route data
    trips_data_set = get_bus_line_data(route_id)
    
    # Get bus data
    bus_data_set = trips_data_set.loc[trips_data_set['block_id'] == bus_id]
    return bus_data_set
    

def print_bus_data(route_id, bus_id):
    '''
    Prints data associated with a particular bus service
    route_id = bus line route Id code (such as 28)
    bus_id   = bus Id code (such as 1308)
    '''
    # Get bus data
    bus_data_set = get_bus_data(route_id, bus_id)

    print("Trips Data Set for route " + route_id + " Bus #" + bus_id + ":")
    print(bus_data_set.to_string(columns=['route_id', 'service_id', 'trip_id', 'trip_headsign', 'block_id', 'shape_id'], 
                                 index=False, justify='left'))

for route_id_list, bus_id_list, trip_id_list in bus_trips:
    route_id = route_id_list[0]
    for bus_id in bus_id_list:
        print_bus_data(route_id, bus_id)
        print()
        print()

Trips Data Set for route 28 Bus #1308:
route_id  service_id  trip_id trip_headsign                      block_id  shape_id
28       10          45049    "Fern Rock Transportation Center"  1308     205690  
28       10          45068    "Fern Rock Transportation Center"  1308     205690  
28       10          45072    "Fern Rock Transportation Center"  1308     205690  
28       10          45076    "Fern Rock Transportation Center"  1308     205690  
28       10          45086                 "Torresdale-Cottman"  1308     205693  
28       10          45108                 "Torresdale-Cottman"  1308     205693  
28       10          45112                 "Torresdale-Cottman"  1308     205693  
28       10          45116                 "Torresdale-Cottman"  1308     205693  
28       11          45120    "Fern Rock Transportation Center"  1308     205690  
28       11          45123    "Fern Rock Transportation Center"  1308     205690  
28       11          45128    "Fern Rock Transp